In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()
Pkg.precompile()

  Activating project at `~/Documents/GitHub/BEE-4750-Final-Project`
   Installed MathOptInterface ─ v1.47.0
Precompiling project...
  37907.9 ms  ✓ MathOptInterface
   3915.9 ms  ✓ MathOptIIS
  13422.9 ms  ✓ HiGHS
  22987.0 ms  ✓ JuMP
  4 dependencies successfully precompiled in 62 seconds. 52 already precompiled.

The following 1 direct dependency failed to precompile:

BEE4750FinalProject 

Error: Missing source file for Base.PkgId(Base.UUID("12345678-1234-1234-1234-123456789012"), "BEE4750FinalProject"  0 dependencies successfully precompiled in 0 seconds. 56 already precompiled.



Base.Precompilation.PkgPrecompileError: The following 1 direct dependency failed to precompile:

BEE4750FinalProject 

Error: Missing source file for Base.PkgId(Base.UUID("12345678-1234-1234-1234-123456789012"), "BEE4750FinalProject"

In [4]:
using JuMP
using HiGHS

#data
T = 1:24 # time steps (24 hours)

#CREATE Q AND D AS ARRAYS SO THAT DEMAND/HEAT VARIES PER HOUR
u = .81 #useable heat from data centers coefficient
C = 1250.0 #average data center capacity in kW

Q_data = fill(C * u, 24) #Data center useable waste heat in kW at each hour
D_th = fill(368.0, 24) # Greenhouse demand in kW at each hour
alpha = 5.0 #L of water per kWh of rejected heat

#Model
model = Model(HiGHS.Optimizer)

#Variables
@variable(model, x_reuse[t in T] >= 0) #heat reused in CEA kW
@variable(model, x_reject[t in T] >= 0) #heat rejected to cooling tower kW

#Objective
@objective(model, Min, sum(alpha * x_reject[t] for t in T))

#Constraints
@constraint(model, [t in T], x_reuse[t] + x_reject[t] == Q_data[t]) #heat balance
@constraint(model, [t in T], x_reuse[t] <= D_th[t]) #reuse limit

optimize!(model)


#print results
println("Optimal total water use in L: ", objective_value(model))

println("\nHour  Reused Energy(kW)  Rejected Energy(kW)")

for t in T
    println(rpad(t, 5), " ",
            rpad(value(x_reuse[t]), 12), " ",
            rpad(value(x_reject[t]), 12))
end


Running HiGHS 1.12.0 (git hash: 755a8e027): Copyright (c) 2025 HiGHS under MIT licence terms
LP has 48 rows; 48 cols; 72 nonzeros
Coefficient ranges:
  Matrix  [1e+00, 1e+00]
  Cost    [5e+00, 5e+00]
  Bound   [0e+00, 0e+00]
  RHS     [4e+02, 1e+03]
Presolving model
0 rows, 0 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve reductions: rows 0(-48); columns 0(-48); nonzeros 0(-72) - Reduced to empty
Performed postsolve
Solving the original LP from the solution after postsolve

Model status        : Optimal
Objective value     :  7.7340000000e+04
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.00
Optimal total water use in L: 77340.00000000001

Hour  Reused Energy(kW)  Rejected Energy(kW)
1     368.0        644.5000000000001
2     368.0        644.5000000000001
3     368.0        644.5000000000001
4     368.0        644.5000000000001
5     368.0        644.5000000000001
6     368.0        644.5000000000001
7     368.0        644.5000000000001
8     36